Installing Dependencies

In [2]:
!pip install flask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.3/101.3 kB 813.1 kB/s eta 0:00:000:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.4 MB/s eta 0:00:0000:0100:01


In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 754.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 959.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 18.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 9.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 18.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.1 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.5 MB/s eta 0:00:00


In [5]:
pip install gtts


Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 1.6 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 6.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 24.5 MB/s eta 0:00:0000:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110721 sha256=2168f146bc8c37ce2205892e5dc078ac5d958dfbe8db3847c65513dd0e74a98e
  Stored in directory: /home/codespace/.cache/pip/wheels/96/32/2d/e10123bd88fbfc02fed53cc18c80a171d3c87479ed845fa7c1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


In [9]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.5 MB/s eta 0:00:0000:0100:01


Importing Dependencies

In [2]:
from flask import Flask, request, jsonify
from transformers import pipeline, set_seed
from gtts import gTTS
from moviepy.editor import *
import torch
from diffusers import StableDiffusionPipeline

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Flask App Setup:

In [3]:
app = Flask(__name__)


Route for Generating Video:

In [4]:
def generate_video():
    data = request.get_json()
    text = data.get('text', 'a girl lost in the woods')
    
    # Generate the story
    generator = pipeline('text-generation', model='gpt2-medium')
    set_seed(42)
    prompt = "generate a story on" + text
    max_length = 1080
    story = generator(prompt, max_new_tokens=max_length, truncation=True)[0]['generated_text']
    
    # Create narration from the story
    narration = gTTS(text=story, lang='en-us', slow=True, tld='com')
    narration.save("narration.mp3")
    audio = AudioFileClip("narration.mp3")
    duration = audio.duration

    # Generate images related to the story
    h = 800
    w = 640
    steps = 25
    guidance = 7.5
    neg = "easynegative,no repetation, lowres,partial view, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worstquality, low quality, normal quality, jpegartifacts, signature, watermark, username, blurry, bad feet, cropped, poorly drawn hands, poorly drawn face, mutation, deformed, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, extra fingers, fewer digits, extra limbs, extra arms,extra legs, malformed limbs, fused fingers, too many fingers, long neck, cross-eyed,mutated hands, polar lowres, bad body, bad proportions, gross proportions, text, error, missing fingers, missing arms, missing legs, extra digit, extra arms, extra leg, extra foot,"
    num_images = int(duration / 10)
    for i in range(num_images):
        prompt = text  # Use the original text as the prompt
        image = pipe(prompt, height=h, width=w, number_inference_steps=steps, guidance_scale=guidance, negative_prompt=neg).images[0]
        image.save(f"image_{i + 1}.png")  # Save the image with a unique name

    # Create a blank video clip with the same duration as the audio
    video = ColorClip((1280, 720), color=(0, 0, 0), duration=duration)

    # Set the text for the video
    txt = story

    # Add the text clip to the video
    video = video.set_audio(audio)
    video = video.set_duration(duration)
    video = video.set_fps(24)
    video = video.set_audio(audio)
    video = video.set_duration(duration)
    video = video.set_fps(24)

    # Add images to the video
    image_clips = [ImageClip(f"image_{i + 1}.png").set_duration(duration / num_images) for i in range(num_images)]
    images_video = concatenate_videoclips(image_clips, method="compose")

    # Overlay the images video on top of the main video
    final_video = CompositeVideoClip([video.set_position(('center', 'center')), images_video.set_position(('center', 'center'))])

    # Write the final video to a file
    final_video_path = "story_video.mp4"
    final_video.write_videofile(final_video_path, codec='libx264', fps=24)

    return jsonify({"message": "Video generated successfully", "video_path": final_video_path})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/home/codespace/.python/current/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/codespace/.python/current/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/codespace/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/codespace/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/codespace/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/codespace/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/codespace/.local/lib/python3.10/site-p

SystemExit: 1